Il sistema è utilizzabile tramite una semplice interfaccia a riga di comando o attraverso un notebook interattivo. Lo scout inserisce liberamente una descrizione del giocatore desiderato e il sistema restituisce automaticamente una shortlist dei migliori candidati, con la possibilità di visualizzare anche i punteggi di similarità o altre informazioni utili alla decisione. In questo modo, il motore di ricerca diventa un vero strumento di supporto alle attività di scouting.

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
import os
import joblib


In [2]:
import os
import re
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics.pairwise import cosine_similarity

# ============================
# 1. Caricamento dataset
# ============================

MODELS_DIR = "../data/models"
DATA_CLEAN_PATH = "../data/drafted_cleaned.csv"

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH     = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH       = os.path.join(MODELS_DIR, "index_metadata.csv")

vectorizer   = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df  = pd.read_csv(META_PATH)

full_df = pd.read_csv(DATA_CLEAN_PATH)
full_df["player_id"] = full_df.index.astype(int)

if "player_id" not in metadata_df.columns:
    metadata_df["player_id"] = metadata_df.index.astype(int)

print("TF-IDF:", tfidf_matrix.shape)
print("Metadata:", metadata_df.shape)
print("Stats DF:", full_df.shape)


# ============================
# 2. Utility
# ============================

def normalize_text(t):
    if not isinstance(t, str):
        return ""
    return re.sub(r"\s+", " ", t.lower()).strip()

def normalize_percentage(v):
    """Normalizza percentuali NBA in modo robusto e coerente."""
    try:
        v = float(v)
    except:
        return None

    # Valori mancanti del database
    if v < 0:
        return None

    # Formato decimale: 0.35 → 35%
    if 0 < v <= 1:
        return v * 100

    # Formato 35 → 35%
    if 1 < v <= 100:
        return v

    # Formato errato es: 350 → 35%
    if v > 100 and v < 1000:
        return v / 10

    # Formato estremo es. 2300 → 23%
    if v >= 1000:
        return v / 100

    return v



# ============================
# 3. BUCKET
# ============================

def bucketize_3p(pct):
    v = normalize_percentage(pct)
    if v is None: return None
    if v >= 40: return "ottimo tiratore da 3 punti"
    if v >= 35: return "buon tiratore da 3 punti"
    if v >= 30: return "tiratore da 3 punti discreto"
    return "tiratore da 3 punti poco affidabile"

def bucketize_ft(pct):
    v = normalize_percentage(pct)
    if v is None: return None
    if v <= 1: v *= 100
    if v >= 85: return "eccellente tiratore di tiri liberi"
    if v >= 75: return "buon tiratore di tiri liberi"
    return "tiratore di liberi migliorabile"

def bucketize_reb(v):
    if v is None: return None
    v = float(v)
    if v >= 9: return "forte rimbalzista"
    if v >= 7: return "buon rimbalzista"
    if v >= 4: return "rimbalzista discreto"
    return "rimbalzista non di impatto"

def bucketize_ast(v):
    if v is None: return None
    v = float(v)
    if v >= 9: return "ottimo playmaker e passatore"
    if v >= 6: return "buon passatore"
    if v >= 3: return "buon assistman"
    if v >= 1: return "assistman discreto"
    return "non particolarmente orientato agli assist"

def bucketize_scorer(v):
    if v is None: return None
    v = float(v)
    if v >= 25: return "realizzatore di altissimo livello"
    if v >= 18: return "buon realizzatore"
    if v >= 12: return "realizzatore discreto"
    return "giocatore non focalizzato sulla realizzazione"


# ============================
# 4. Pattern con regex corrette
# ============================

SKILL_PATTERNS = {
    "shooting_3": [
        r"3 ?punti", r"da 3", r"tiro da 3", r"tripla", r"percentuali.*3", r"3p"
    ],
    "ft": [
        r"tiri liberi", r"liberi", r"ft"
    ],
    "reb": [
        r"rimbalzi", r"rimbalzista"
    ],
    "playmaking": [
        r"assist", r"passatore", r"playmaker"
    ],
    "scorer": [
        r"realizzatore", r"tanti punti", r"prima opzione"
    ]
}

# Regex che cattura TUTTI i casi di "ritirato" → ritirato, ritirata, ritirati, ritirato.
STATUS_PATTERNS = {
    "Active": [
        r"in attività", r"ancora attivo", r"gioca ancora"
    ],
    "Retired": [
        r"ritirat\w*",        # <-- MATCH universale: ritirato, ritirato., ritirati ecc.
        r"non gioca più",
        r"ha smesso"
    ]
}


# ============================
# 5. Parsing intenti
# ============================

def parse_query_intents(q):
    q_norm = normalize_text(q)
    skills = set()
    status = None

    # stato
    for st, pats in STATUS_PATTERNS.items():
        if any(re.search(p, q_norm) for p in pats):
            status = st

    # skill
    for sk, pats in SKILL_PATTERNS.items():
        if any(re.search(p, q_norm) for p in pats):
            skills.add(sk)

    return {
        "normalized_query": q_norm,
        "skills": sorted(list(skills)),
        "status": status
    }


# ============================
# 6. Query pesata
# ============================

SEMANTIC_EXPANSIONS = {
    "shooting_3": "ottimo tiratore da 3 3 punti 3P% buon tiratore da 3",
    "ft": "tiri liberi FT% buon tiratore ai liberi",
    "reb": "rimbalzi forte rimbalzista",
    "playmaking": "assist buon passatore playmaker",
    "scorer": "punti realizzatore"
}

SEMANTIC_WEIGHTS = {
    "shooting_3": 4, "ft": 3, "reb": 3,
    "playmaking": 3, "scorer": 3
}


def build_weighted_query(q, intents):
    base = intents["normalized_query"]
    parts = [base]
    for sk in intents["skills"]:
        exp = SEMANTIC_EXPANSIONS.get(sk, "")
        w = SEMANTIC_WEIGHTS.get(sk, 1)
        parts.append((" " + exp) * w)
    return " ".join(parts).strip()


# ============================
# 7. Motore IR
# ============================

def search_players_scout(query, top_k=2000):
    intents = parse_query_intents(query)
    weighted = build_weighted_query(query, intents)

    vec = vectorizer.transform([weighted])
    sim = cosine_similarity(vec, tfidf_matrix).ravel()

    idx = np.argsort(sim)[::-1]
    res = metadata_df.iloc[idx].copy()
    res["similarity"] = sim[idx]
    res.attrs["intents"] = intents
    return res.head(top_k)


# ============================
# 8. Rank + Explain + filtro AND
# ============================

def rank_and_explain(query, top_k=10):

    # 1) Ricerca iniziale testuale (TF-IDF)
    base = search_players_scout(query)
    intents = base.attrs["intents"]

    # 2) Merge con dataset completo
    merged = base.merge(full_df, on="player_id", how="left")

    # ============================
    # 3) GENERAZIONE BUCKETS 
    # ============================
    merged["bucket_3p"] = merged["3P%"].apply(bucketize_3p)
    merged["bucket_ft"] = merged["FT%"].apply(bucketize_ft)
    merged["bucket_reb"] = merged["TRB.1"].apply(bucketize_reb)
    merged["bucket_ast"] = merged["AST.1"].apply(bucketize_ast)
    merged["bucket_pts"] = merged["PTS.1"].apply(bucketize_scorer)

    # ============================
    # 4) FILTRO STATO (Active/Retired)
    # ============================
    if intents["status"]:
        status_cols = [c for c in merged.columns if c.startswith("Status")]
        stcol = status_cols[0]
        merged = merged[merged[stcol] == intents["status"]]

    # ============================
    # 5) FILTRO SKILLS (solo quelli che soddisfano la skill)
    # ============================
    for sk in intents["skills"]:
        # Mappa la skill sul nome della colonna bucket corretta
        col = f"bucket_{sk.split('_')[0]}"
        col = col.replace("bucket_shooting", "bucket_3p")
        if col in merged.columns:
            merged = merged[merged[col].notna()]

    # ============================
    # 6) RANKING STATISTICO SULLE SKILL
    # ============================

    skills = intents.get("skills", [])

    # Rimbalzi → ordina per TRB.1 (per partita)
    if "reb" in skills and "TRB.1" in merged.columns:
        merged = merged.sort_values(by="TRB.1", ascending=False)

    # Assist → ordina per AST.1
    if "playmaking" in skills and "AST.1" in merged.columns:
        merged = merged.sort_values(by="AST.1", ascending=False)

    # Punti → ordina per PTS.1
    if "scorer" in skills and "PTS.1" in merged.columns:
        merged = merged.sort_values(by="PTS.1", ascending=False)

    # Tiro da 3 → ordina per 3P%
    if "shooting_3" in skills and "3P%" in merged.columns:
        merged = merged.sort_values(by="3P%", ascending=False)

    # Tiri liberi → ordina per FT%
    if "ft" in skills and "FT%" in merged.columns:
        merged = merged.sort_values(by="FT%", ascending=False)

    # ============================
    # 7) RITORNA TOP-K
    # ============================
    return merged.head(top_k), intents



def fmt_pct(x):
    if pd.isna(x): return None
    x = float(x)
    if x < 1: x *= 100
    return f"{x:.1f}%"

def fmt_num(x):
    if pd.isna(x): return None
    return f"{float(x):.1f}"


def print_formatted_result(row, intents, idx):

    name = row.get("Player_x") or row.get("Player_y") or row["Player"]

    print(f"\n#{idx+1} — {name}")
    print("-" * (len(name) + 6))

    if "shooting_3" in intents["skills"]:
        print(f"- {row['bucket_3p']} ({fmt_pct(row['3P%'])})")

    if "ft" in intents["skills"]:
        print(f"- {row['bucket_ft']} ({fmt_pct(row['FT%'])})")

    if "reb" in intents["skills"]:
        print(f"- {row['bucket_reb']} ({fmt_num(row['TRB.1'])} rimbalzi)")

    if "playmaking" in intents["skills"]:
        print(f"- {row['bucket_ast']} ({fmt_num(row['AST.1'])} assist)")

    if "scorer" in intents["skills"]:
        print(f"- {row['bucket_pts']} ({fmt_num(row['PTS.1'])} punti)")


def run_talent_scout_cli_table(top_k=10):
    print("=== SISTEMA INTELLIGENTE PER TALENT SCOUT NBA ===")
    print("Scrivi una descrizione del giocatore che cerchi.")
    print("Digita 'exit' per uscire.\n")

    while True:
        query = input("\nQuery: ").strip()
        if query.lower() in ("exit", "esci", "quit"):
            print("Chiusura sistema 👋")
            break

        results, intents = rank_and_explain(query, top_k)

        # Format numerici
        results["3P_value"]  = results["3P%"].apply(fmt_pct)
        results["FT_value"]  = results["FT%"].apply(fmt_pct)
        results["AST_value"] = results["AST.1"].apply(fmt_num)
        results["REB_value"] = results["TRB.1"].apply(fmt_num)
        results["PTS_value"] = results["PTS.1"].apply(fmt_num)

        # Costruzione colonne dinamiche
        display_cols = ["Player_x", "similarity"]

        if "shooting_3" in intents["skills"]:
            display_cols += ["bucket_3p", "3P_value"]

        if "ft" in intents["skills"]:
            display_cols += ["bucket_ft", "FT_value"]

        if "playmaking" in intents["skills"]:
            display_cols += ["bucket_ast", "AST_value"]

        if "reb" in intents["skills"]:
            display_cols += ["bucket_reb", "REB_value"]

        if "scorer" in intents["skills"]:
            display_cols += ["bucket_pts", "PTS_value"]

        print("\n📝 Query:", query)
        print("🔍 Skill individuate:", intents["skills"])
        print("🔍 Stato:", intents["status"], "\n")

        # MOSTRARE TABELLA
        display(results[display_cols])




TF-IDF: (8323, 22019)
Metadata: (8323, 7)
Stats DF: (8323, 37)


In [3]:
run_talent_scout_cli_table(top_k=10)

=== SISTEMA INTELLIGENTE PER TALENT SCOUT NBA ===
Scrivi una descrizione del giocatore che cerchi.
Digita 'exit' per uscire.


📝 Query: 
🔍 Skill individuate: []
🔍 Stato: None 



,Player_x,similarity
0,Ariel Hukporti,0.0
1,Ulrich Chomche,0.0
2,Kevin McCullar Jr.,0.0
3,Bronny James,0.0
4,Anton Watson,0.0
5,Cam Spencer,0.0
6,Quinten Post,0.0
7,Melvin Ajinça,0.0
8,Enrique Freeman,0.0
9,Tristen Newton,0.0


Chiusura sistema 👋
